# 定量評価

- テストデータ自体のDistinctの値
- 同一入力に対するモデルの出力の結果サンプルを載せる
- 生成文がPOSに沿っている率を載せる

## データ読み込み
- デコードは `inference-vae-pos` で行う

In [3]:
import init
import os

In [6]:
import pandas as pd

In [68]:
posvae_decoded_path = '/root/user/work/src/Sentence-VAE/runs/posvae_eccos_v2_TS=2020-01-05-173701_BS=32_LR=0.001_EB=300_GRU_HS=256_L=1_BI=0_LS=16_WD=0_ANN=LOGISTIC_K=0.0025_X0=2500/dumps/posvae_eccos_all.pickle'
vae_decoded_path = '/root/user/work/src/Sentence-VAE/runs/vae_eccos_v2_TS=2020-01-05-181436_BS=32_LR=0.001_EB=300_GRU_HS=256_L=1_BI=0_LS=16_WD=0_ANN=LOGISTIC_K=0.0025_X0=2500/dumps/vae_eccos_all.pickle'

In [69]:
df_posvae_decoded = pd.read_pickle(posvae_decoded_path)
df_vae_decoded = pd.read_pickle(vae_decoded_path)

In [70]:
text_cols = [col for col in df_decoded.columns if '生成文' in col and 'ids' not in col]
id_cols = [col for col in df_decoded.columns if '生成文_ids' in col]

In [71]:
multi_posvae_decoded_ids = [[v for k,v in row.items()] for row in df_posvae_decoded[id_cols].to_dict(orient='records')]
multi_vae_decoded_ids = [[v for k,v in row.items()] for row in df_vae_decoded[id_cols].to_dict(orient='records')]

In [73]:
assert len(multi_posvae_decoded_ids) == len(multi_vae_decoded_ids)
assert len(multi_posvae_decoded_ids[0]) == len(multi_vae_decoded_ids[0])
len(multi_posvae_decoded_ids), len(multi_posvae_decoded_ids[0])

(2479, 10)

# Local-Distinctの計測

In [74]:
from metric import flat_and_calculate_distinct
def calculate_local_distinct(multi_ids, ngrams=[1, 2, 3,]):
    # Distinct-1, 2, full
    distinct_dict_list = []
    for ids in multi_ids:
        distinct_dict, _ = flat_and_calculate_distinct(ids, ngrams)
        distinct_dict_list.append(distinct_dict)
    distinct_dict = pd.DataFrame(distinct_dict_list).mean().to_dict()
    distinct_dict = {f'local_{k}': v for k,v in distinct_dict.items() if 'distinct' in k}
    return distinct_dict, distinct_dict_list

In [76]:
distinct_posvae_dict, distinct_posvae_dict_list = calculate_local_distinct(multi_posvae_decoded_ids)
df_posvae_distinct = pd.DataFrame(distinct_posvae_dict_list)

distinct_vae_dict, distinct_vae_dict_list = calculate_local_distinct(multi_vae_decoded_ids)
df_vae_distinct = pd.DataFrame(distinct_vae_dict_list)

In [78]:
distinct_posvae_dict, distinct_vae_dict

({'local_distinct_1gram': 0.3969541020241795,
  'local_distinct_2gram': 0.7122047906786498,
  'local_distinct_3gram': 0.8361804480148379},
 {'local_distinct_1gram': 0.4069951986330746,
  'local_distinct_2gram': 0.7056350056568841,
  'local_distinct_3gram': 0.8109642806148886})

In [62]:
assert df_distinct.shape[0] == df_decoded.shape[0]

In [96]:
dist_cols = 'distinct_1gram distinct_2gram distinct_3gram'.split(' ')
df_posvae = pd.merge(df_posvae_decoded, df_posvae_distinct, left_index=True, right_index=True)[['入力文'] + text_cols + dist_cols]
df_posvae = df_posvae.rename(columns={col: f'posvae_{col}' for col in df_posvae.columns if '入力文' not in col})

df_vae = pd.merge(df_vae_decoded, df_vae_distinct, left_index=True, right_index=True)[['入力文'] + text_cols + dist_cols]
df_vae = df_vae.rename(columns={col: f'vae_{col}' for col in df_vae.columns if '入力文' not in col})

In [100]:
df = pd.merge(df_vae, df_posvae, on='入力文')

In [147]:
df[:1].T

,0
入力文,<num>代女性が<num>代に見えるのは、「シワに貼って寝るだけ!」
vae_生成文0,<num>歳に見える<num>歳主婦は「目元のシワ」が<num>秒でアイロンみたいに!?
vae_生成文1,「目元のたるみ」には菌を塗るだけ!<num>円で試せる!
vae_生成文2,<num>代に人気の「シワ」がない理由!<num>割以上が誤解した方法がスゴい
vae_生成文3,<num>歳に見える<num>歳主婦は「再生医療センター」がない理由!<num>歳主婦が<num>代に間違えられた美容液
vae_生成文4,<num>歳に見える<num>歳主婦「ほうれい線」がない理由!<num>歳主婦が<num>日でピンッ
vae_生成文5,<num>代に見える<num>歳主婦は「再生医療美容液」が話題!
vae_生成文6,<num>代に見える<num>歳主婦は「あるものを塗る」と言われる程のようなハリ肌に
vae_生成文7,<num>歳に見える<num>歳主婦は「再生医療美容液」が話題に!<num>円で試せる!
vae_生成文8,「<num>歳に見える<num>歳は?」おばさん顔の素を<num>日で卒業


# Latex用出力

In [139]:
def tex_escape(text):
    return text.replace('<', r'\verb|<|').replace('>', r'\verb|>|').replace('%', r'％')

In [141]:
_id = 754
# decoded_texts = df.iloc[_id][[c for c in df.columns if 'posvae_生成' in c]].tolist()
decoded_texts = df.iloc[_id][[c for c in df.columns if 'vae_生成' in c and 'posvae' not in c]].tolist()
print('\n'.join(decoded_texts))

皮膚科医(<num>)「シミは家で消せるよ?」簡単すぎる方法が大炎上中
【シミはレーザー治療?】皮膚科医(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
【手遅れのシミは簡単に消える】皮膚科医(<num>)「シミは家で消せるよ」簡単すぎる方法が大炎上中
【レーザー治療はもう古い?】皮膚科医の妻(<num>)「シミは家で簡単に消えるよ?」効果が凄すぎて大炎上中
【ファンデ不要】スッピン美人急増中!シミの悩みにはこれ
【シミ・シワ・たるみ・シワ・たるみ・たるみ・たるみ】皮膚科医の妻(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
【シミ・肝斑・肝斑】皮膚科医の妻(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
【シミ・肝斑・肝斑】皮膚科医の妻(<num>)「シミは家で簡単に消えるよ?」簡単すぎて大炎上中
【手遅れのシミは簡単に消える】皮膚科医の妻(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
【シミ対策はもう古い?】杉山愛さんのシミ対策


In [142]:
print('\\\\\n'.join([tex_escape(text) for text in decoded_texts]))

皮膚科医(\verb|<|num\verb|>|)「シミは家で消せるよ?」簡単すぎる方法が大炎上中\\
【シミはレーザー治療?】皮膚科医(\verb|<|num\verb|>|)「シミは家で消せるよ?」簡単すぎて大炎上中\\
【手遅れのシミは簡単に消える】皮膚科医(\verb|<|num\verb|>|)「シミは家で消せるよ」簡単すぎる方法が大炎上中\\
【レーザー治療はもう古い?】皮膚科医の妻(\verb|<|num\verb|>|)「シミは家で簡単に消えるよ?」効果が凄すぎて大炎上中\\
【ファンデ不要】スッピン美人急増中!シミの悩みにはこれ\\
【シミ・シワ・たるみ・シワ・たるみ・たるみ・たるみ】皮膚科医の妻(\verb|<|num\verb|>|)「シミは家で消せるよ?」簡単すぎて大炎上中\\
【シミ・肝斑・肝斑】皮膚科医の妻(\verb|<|num\verb|>|)「シミは家で消せるよ?」簡単すぎて大炎上中\\
【シミ・肝斑・肝斑】皮膚科医の妻(\verb|<|num\verb|>|)「シミは家で簡単に消えるよ?」簡単すぎて大炎上中\\
【手遅れのシミは簡単に消える】皮膚科医の妻(\verb|<|num\verb|>|)「シミは家で消せるよ?」簡単すぎて大炎上中\\
【シミ対策はもう古い?】杉山愛さんのシミ対策


In [116]:
df.iloc[754].to_frame()

,754
入力文,【レーザーじゃないシミ対策】デキる美魔女はみんなやってる簡単自宅ケア?
vae_生成文0,皮膚科医(<num>)「シミは家で消せるよ?」簡単すぎる方法が大炎上中
vae_生成文1,【シミはレーザー治療?】皮膚科医(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
vae_生成文2,【手遅れのシミは簡単に消える】皮膚科医(<num>)「シミは家で消せるよ」簡単すぎる方法が大炎上中
vae_生成文3,【レーザー治療はもう古い?】皮膚科医の妻(<num>)「シミは家で簡単に消えるよ?」効果が凄すぎて大炎上中
vae_生成文4,【ファンデ不要】スッピン美人急増中!シミの悩みにはこれ
vae_生成文5,【シミ・シワ・たるみ・シワ・たるみ・たるみ・たるみ】皮膚科医の妻(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
vae_生成文6,【シミ・肝斑・肝斑】皮膚科医の妻(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中
vae_生成文7,【シミ・肝斑・肝斑】皮膚科医の妻(<num>)「シミは家で簡単に消えるよ?」簡単すぎて大炎上中
vae_生成文8,【手遅れのシミは簡単に消える】皮膚科医の妻(<num>)「シミは家で消せるよ?」簡単すぎて大炎上中


## POS-VAE が有利なレコード

In [ ]:
df['posvae_vae_distinct_1gram'] = df.posvae_distinct_1gram - df.vae_distinct_1gram
df['posvae_vae_distinct_2gram'] = df.posvae_distinct_2gram - df.vae_distinct_2gram
df['posvae_vae_distinct_3gram'] = df.posvae_distinct_3gram - df.vae_distinct_3gram

In [145]:
# チェリーピッキング用： 最もPOS-VAEが有利なもの順に並べる
# df.sort_values('posvae_vae_distinct_3gram', ascending=False)[:100]